In [1]:
import pandas as pd
import os 
import sys

In [2]:
current_dir = os.getcwd()
funcs_dir = os.path.join(current_dir, 'funcs')
if funcs_dir not in sys.path:
    sys.path.append(funcs_dir)

In [3]:
from funcs.transform_row_to_turns import transform_row_to_turns
from funcs.load_id_to_name_mapping import load_id_to_name_mapping
from funcs.load_id_to_wr_mappings import load_id_to_wr_mappings
from funcs.display_game_state import display_game_state

In [4]:
# import pickled model ../models/win_probability/xgb_win_prob_model.pkl
import pickle
model_path = os.path.join('..', 'models', 'win_probability', 'xgb_win_prob_model.pkl')
with open(model_path, 'rb') as f:
    model = pickle.load(f)



In [13]:
data = pd.read_csv('../data/replay_data/replay_data_public.DSK.PremierDraft.csv', nrows = 1000)

C:\Users\Jack Wright\AppData\Local\Temp\ipykernel_14716\1330376792.py:1: DtypeWarning: Columns (20,92,116,117,121,126,128,130,136,146,147,155,159,163,166,182,185,188,193,198,215,218,221,223,228,229,231,244,245,253,259,260,276,306,307,323,337,338,343,355,368,369,372,385,400,401,405,416,417,430,431,434,447,462,463,467,479,492,496,509,523,524,525,528,529,540,541,554,586,587,589,590,593,603,616,619,620,631,632,633,648,651,652,655,662,663,665,681,683,694,710,712,713,714,717,724,726,740,743,745,746,755,774,776,779,781,786,787,789,804,805,808,811,812,813,816,817,836,837,843,845,849,851,864,866,867,869,870,872,873,874,875,879,894,896,903,905,906,907,923,929,931,934,935,937,940,941,956,964,965,966,968,985,990,996,997,999,1008,1015,1018,1026,1028,1030,1040,1044,1045,1047,1052,1056,1058,1059,1061,1064,1070,1077,1080,1088,1090,1104,1105,1106,1107,1108,1109,1114,1119,1132,1136,1137,1139,1142,1147,1150,1152,1154,1164,1166,1167,1168,1169,1170,1171,1181,1182,1185,1194,1196,1197,1198,1199,1200,1201,120

In [6]:

def clear_screen():
    """
    Clears the terminal screen.
    Works on Windows, macOS, and Linux.
    """
    os.system('cls' if os.name == 'nt' else 'clear')

def watch_game(all_turns):
    """
    Displays game states sequentially, clearing the screen between states.
    - all_turns: list of game states (DataFrames or Series).
    """
    if not all_turns:
        print("No game states to display.")
        return

    for idx, turn in enumerate(all_turns):
        clear_screen()  # Clear the screen
        print(f"Displaying game state {idx + 1} of {len(all_turns)}:\n")
        display_game_state(turn)  # Display the current game state
        input("\nPress Enter to continue...")  # Wait for user to press Enter

In [11]:
# grab a row which is one whole game
raw_game_row = data.iloc[10]
# transform this row into a dataframe for prediction and display
turn_df_calc = transform_row_to_turns(raw_game_row, display = False,  max_cards=20)
# Transform the row into a turn-level DataFrame
turn_df_calc = transform_row_to_turns(raw_game_row, display=False, max_cards=20)

# drop the 'game_id' and 'won' columns for prediction
# Drop the 'game_id' column as it is non-numeric and not needed for predictions
turn_df_calc = turn_df_calc.drop(['game_id','won'], axis=1)

predict_proba = model.predict_proba(turn_df_calc)


# create the display dataframe

df_turn_display = transform_row_to_turns(raw_game_row, display=True, max_cards=20)

# append the predict_proba to df_turn_display
df_turn_display['predict_proba'] = predict_proba[:,1]


# # Iterate through all rows in all_turns_df
# for index, row in df_turn_display.iterrows():
#     clear_screen()  # Clear the screen
#     print(f"Displaying game state {index + 1} of {len(df_turn_display)}:\n")
#     display_game_state(row)  # Pass the row as a Series
#     input("\nPress Enter to continue...")


C:\Users\Jack Wright\AppData\Local\Temp\ipykernel_14716\772885739.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_turn_display['predict_proba'] = predict_proba[:,1]


In [18]:
def predict_and_display_game(data, model, game_row_index, max_cards=20):
    """
    Predicts win probabilities for a single game, displays the game state turn by turn,
    and appends predictions for user review.
    
    Parameters:
        data (pd.DataFrame): Full dataset containing game data.
        model: Trained model used for predicting probabilities.
        game_row_index (int): Index of the game to process and display.
        max_cards (int): Maximum number of card slots for processing.
    """
    # Grab the row representing one whole game
    raw_game_row = data.iloc[game_row_index]
    
    # Transform the row into a DataFrame for prediction
    turn_df_calc = transform_row_to_turns(raw_game_row, display=False, max_cards=max_cards)

    # Drop non-numeric columns for prediction
    turn_df_calc = turn_df_calc.drop(['game_id', 'won'], axis=1)

    # Predict probabilities
    predict_proba = model.predict_proba(turn_df_calc)

    # Create the display DataFrame with card names
    df_turn_display = transform_row_to_turns(raw_game_row, display=True, max_cards=max_cards)

    # Append the predicted probabilities to the display DataFrame
    df_turn_display['predict_proba'] = predict_proba[:, 1]

    # Iterate through all turns in the display DataFrame
    for index, row in df_turn_display.iterrows():
        clear_screen()  # Clear the screen
        print(f"Displaying game state {index + 1} of {len(df_turn_display)}:\n")
        display_game_state(row)  # Pass the row as a Series
        input("\nPress Enter to continue...")

    # Final Turn Analysis and Display
    final_row = df_turn_display.iloc[-1]
    user_life = final_row['user_life']
    oppo_life = final_row['oppo_life']
    final_probability = final_row['predict_proba']
    result = "User Won" if user_life > oppo_life else "Opponent Won"
    outcome = "FATALITY" if user_life == 0 or oppo_life == 0 else "CONCESSION"

    # Create the cool graphic
    clear_screen()
    print("=" * 50)
    print(f"{'FINAL GAME STATE':^50}")
    print("=" * 50)
    print(f"{'Result:':<20} {result}")
    print(f"{'Final Win Probability:':<20} {final_probability:.2%}")
    print(f"{'User Life:':<20} {user_life}")
    print(f"{'Opponent Life:':<20} {oppo_life}")
    print(f"{'Outcome:':<20} {outcome}")
    print("=" * 50)


In [20]:
predict_and_display_game(data, model, 550)

C:\Users\Jack Wright\AppData\Local\Temp\ipykernel_14716\4031454411.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_turn_display['predict_proba'] = predict_proba[:, 1]


Displaying game state 1 of 10:

Predicted Win Probability: 0.6048962473869324

******************************

Turn 1
Opponent Life: 20.0
Opponent Hand:
[ ??? ] [ ??? ] [ ??? ] [ ??? ] [ ??? ] [ ??? ] [ ??? ]
Opponent Lands:
[Land ]
Opponent Board:
  ||  


  ||  
User Board:
[Land ]
User Lands:
[Oblivious Bookworm] [Lakeside Shack] [Spineseeker Centipede] [Forest] [Say Its Name]
User Hand:
User Life: 20.0
Displaying game state 2 of 10:

Predicted Win Probability: 0.6629887819290161

******************************

Turn 2
Opponent Life: 20.0
Opponent Hand:
[ ??? ] [ ??? ] [ ??? ] [ ??? ] [ ??? ] [ ??? ]
Opponent Lands:
[Land ] [Land ]
Opponent Board:
[Nashi, Searcher in the Dark]  ||  


[Oblivious Bookworm]  ||  
User Board:
[Land ] [Land ]
User Lands:
[Murky Sewer] [Lakeside Shack] [Spineseeker Centipede] [Say Its Name]
User Hand:
User Life: 20.0
Displaying game state 3 of 10:

Predicted Win Probability: 0.624198853969574

******************************

Turn 3
Opponent Life: 20.0
Op